In [10]:
def greet(name):
    "This function returns a greeting"
    return f"Hello, {name}"

greet("Dorothy")

'Hello, Dorothy'

In [3]:
def shout(name):
    return f"HELLO, {name.upper()}"
shout("Dorothy")


'HELLO, DOROTHY'

In [4]:
my_func = greet
my_func("Dorothy")

'Hello, Dorothy'

In [12]:
greet.__name__, greet.__doc__, greet.__module__

('greet', 'This function returns a greeting', '__main__')

In [18]:
greetings = {
    "polite": greet,
    "loud":shout
}
greetings["polite"]("Dorothy")

'Hello, Dorothy'

In [19]:
for style, func in greetings.items():
    print(f"{style}: {func("Dorothy")}")

polite: Hello, Dorothy
loud: HELLO, DOROTHY


In [21]:
double = (lambda x: x+x)
double(2)

4

In [23]:
# // function double(x) {
# //     return x+x
# // }
# // double(x)

# // double = function (x) {
# //     return x+x
# // }

In [24]:
def apply_function(func, value):
    return func(value)
apply_function(greet, "Dorothy")

'Hello, Dorothy'

In [25]:
def repeat_twice(func, value):
    return func(value) + func(value)
repeat_twice(greet, "Dorothy")

'Hello, DorothyHello, Dorothy'

In [26]:
def make_echo_version(func):
    def echo_version(arg):
        return (func(arg),func(arg))
    return echo_version

greet_echo = make_echo_version(greet)
greet_echo("Dorothy")


('Hello, Dorothy', 'Hello, Dorothy')

In [27]:
double_echo = make_echo_version(double)
double_echo(4)

(8, 8)

In [ ]:
def make_echo_version(func):
    def echo_version(*args, **kwargs):
        return (func(*args, **kwargs),func(*args, **kwargs))
    return echo_version

def add(x,y,emphasis=False):
    if emphasis:
        return str(x+y)+"!"
    return x+y

add_echo = make_echo_version(add)
add_echo(2,3,emphasis=True)

('5!', '5!')

In [32]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} took {end_time - start_time:.4f} seconds")
        return result
    return wrapper

def slow_function():
    """A function that takes some time to execute."""
    time.sleep(1)
    return "Done!"

slow_function_timer = timer(slow_function)
result = slow_function_timer()
result




slow_function took 1.0001 seconds


'Done!'

In [33]:
import time
import functools

def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} took {end_time - start_time:.4f} seconds")
        return result
    return wrapper

import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} took {end_time - start_time:.4f} seconds")
        return result
    return wrapper

@timer
def slow_function2():
    """A function that takes some time to execute."""
    time.sleep(1)
    return "Done!"

result = slow_function2()
result

slow_function2 took 1.0001 seconds


'Done!'

In [38]:
def bold(func):
    """Wrap result in bold tags."""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return f"<b>{result}</b>"
    return wrapper

def italic(func):
    """Wrap result in italic tags."""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return f"<i>{result}</i>"
    return wrapper

@italic
@bold
def say_hi():
    return "Hello!"

say_hi()

'<i><b>Hello!</b></i>'

In [39]:
def say_hello(name):
    return f"Hello, {name}!"
say_hello("Alice")  # say_hello has been called 1 times
say_hello("Bob")    # say_hello has been called 2 times
say_hello("Charlie") # say_hello has been called 3 times


'Hello, Charlie!'

In [40]:
def counter(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.calls += 1
        print(f"{func.__name__} has been called {wrapper.calls} times")
        return func(*args, **kwargs)
    wrapper.calls = 0
    return wrapper

@counter
def say_hello2(name):
    return f"Hello, {name}!"
    
say_hello2("Alice")  # say_hello has been called 1 times
say_hello2("Bob")    # say_hello has been called 2 times
say_hello2("Charlie") # say_hello has been called 3 times

say_hello2 has been called 1 times
say_hello2 has been called 2 times
say_hello2 has been called 3 times


'Hello, Charlie!'

In [58]:
def retry(max_attempts=3, delay=1):
    """Decorator that retries a function on failure."""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    if attempt < max_attempts - 1:
                        print(f"Attempt {attempt + 1} failed: {e}. Retrying in {delay}s...")
                        time.sleep(delay)
                    else:
                        print(f"All {max_attempts} attempts failed.")
            
            raise last_exception
        return wrapper
    return decorator

@retry(max_attempts=5)
def say_hello3(name):
    import random
    if random.random() < 0.4:  # 40% chance of failure
        raise ConnectionError("Network error")
    return f"Hello, {name}!"

say_hello3("Alice")

Attempt 1 failed: Network error. Retrying in 1s...


'Hello, Alice!'

In [63]:
def validate(*validators):
    """Decorator that validates function arguments."""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # Validate positional arguments
            for i, (arg, validator) in enumerate(zip(args, validators)):
                if not validator(arg):
                    raise ValueError(f"Argument {i} failed validation: {arg}")
            
            return func(*args, **kwargs)
        return wrapper
    return decorator

# Validator functions
def is_positive(x):
    return isinstance(x, (int, float)) and x > 0

def is_string(x):
    return isinstance(x, str) and len(x) > 0

@validate(is_positive, is_string)
def create_user(age, name):
    """Create a user with validated input."""
    return f"User {name}, age {age}"

# Valid usage
user1 = create_user(25, "")
print(user1)  # User Alice, age 25

# Invalid usage would raise ValueError
# user2 = create_user(-5, "Bob")    # Negative age
# user3 = create_user(30, "")       # Empty name


ValueError: Argument 1 failed validation: 

In [65]:
def add_repr(cls):
    def __repr__(self):
        class_name = self.__class__.__name__
        attrs = ', '.join(f"{k}={v!r}" for k, v in self.__dict__.items())
        return f"{class_name}({attrs})"
    cls.__repr__ = __repr__
    return cls

@add_repr
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(10,20)
p

Point(x=10, y=20)

In [67]:
@add_repr
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
person = Person("Charlotte",4)
person

Person(name='Charlotte', age=4)

In [68]:
import inspect

def analyze_function(func):
    """Analyze and display function metadata."""
    print(f"Function: {func.__name__}")
    print(f"Module: {func.__module__}")
    print(f"Doc: {func.__doc__}")
    
    # Get function signature
    sig = inspect.signature(func)
    print(f"Signature: {sig}")
    
    # Analyze parameters
    for name, param in sig.parameters.items():
        print(f"  Parameter '{name}':")
        print(f"    Kind: {param.kind}")
        print(f"    Default: {param.default}")
        print(f"    Annotation: {param.annotation}")
    
    # Return annotation
    if sig.return_annotation != inspect.Signature.empty:
        print(f"Return annotation: {sig.return_annotation}")

def sample_function(a: int, b: str = "default", *args, **kwargs) -> str:
    """A sample function for inspection."""
    return f"{a}: {b}"

analyze_function(sample_function)


Function: sample_function
Module: __main__
Doc: A sample function for inspection.
Signature: (a: int, b: str = 'default', *args, **kwargs) -> str
  Parameter 'a':
    Kind: POSITIONAL_OR_KEYWORD
    Default: <class 'inspect._empty'>
    Annotation: <class 'int'>
  Parameter 'b':
    Kind: POSITIONAL_OR_KEYWORD
    Default: default
    Annotation: <class 'str'>
  Parameter 'args':
    Kind: VAR_POSITIONAL
    Default: <class 'inspect._empty'>
    Annotation: <class 'inspect._empty'>
  Parameter 'kwargs':
    Kind: VAR_KEYWORD
    Default: <class 'inspect._empty'>
    Annotation: <class 'inspect._empty'>
Return annotation: <class 'str'>


In [70]:
def debug_trace():
    """Print debug information about the current call stack."""
    frame = inspect.currentframe()
    
    try:
        # Get the caller's frame
        caller_frame = frame.f_back
        
        print("Debug trace:")
        print(f"  Function: {caller_frame.f_code.co_name}")
        print(f"  File: {caller_frame.f_code.co_filename}")
        print(f"  Line: {caller_frame.f_lineno}")
        print(f"  Local variables: {caller_frame.f_locals}")
        
        # Walk up the call stack
        current_frame = caller_frame
        level = 1
        
        while current_frame.f_back and level < 3:
            current_frame = current_frame.f_back
            print(f"  Caller {level}: {current_frame.f_code.co_name} "
                  f"at line {current_frame.f_lineno}")
            level += 1
            
    finally:
        del frame  # Prevent reference cycles

def business_logic(x, y):
    """Some business logic that might need debugging."""
    intermediate = x * 2
    debug_trace()  # Call our debug function
    return intermediate + y

def business_logic_preamble(x, y):
    """Some business logic that might need debugging."""
    intermediate = x * 2
    business_logic(x, intermediate)
    return intermediate + y


def main():
    result = business_logic_preamble(5, 3)
    print(f"Result: {result}")

main()


Debug trace:
  Function: business_logic
  File: /tmp/ipykernel_5776/3487965858.py
  Line: 31
  Local variables: {'x': 5, 'y': 10, 'intermediate': 10}
  Caller 1: business_logic_preamble at line 37
  Caller 2: main at line 42
Result: 13
